In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Subset
import torch.nn.functional as F
import numpy as np

## Saving a classifier

In this notebook, we will use the classifier that you built in p1.

Hence, first go to that notebook and _export_ the classifier you built there, by adding the following code in that notebook:


In [ ]:
#torch.save(model.state_dict(), name)

In [ ]:
class LeNet(nn.Module):
    def __init__(self, calibrated=False):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, 1, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5, 1)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*16) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)



## Loading a pre-trained classifier

Now, we can load that pre-trained classifier in this notebook as follows:

In [ ]:

def load_clf(clf_classname, path):
    net = clf_classname()
    state_dict = torch.load(path, map_location=lambda storage, loc: storage)
    net.load_state_dict(state_dict)
    return net

model = load_clf(LeNet, 'LeNet.pt')

In [ ]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('.', download=True, train=True, transform=transform)
testset = datasets.MNIST('.', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True)

## Recap: solving a sudoku based on the predictions

In the following, we repeat the code of the previous notebook for sampling a sudoku and getting predictions.

We also included example _ortools_ code that solves the sudoku problem _(requires to install ortools, e.g. conda install ortools)_

In [ ]:
# sample a dataset index with that value/label
def sample_by_label(labels, value):
    # primitive but it works...
    idxs = torch.randperm(len(labels))
    for idx in idxs:
        if labels[idx] == value:
            return idx
# sample a dataset index for each non-zero number
def sample_visual_sudoku(sudoku_p, loader):
    for (images, labels) in loader: # sample one batch
        nonzero = sudoku_p > 0
        vizsudoku = torch.zeros((9,9,1,28,28), dtype=images.dtype)
        idxs = torch.LongTensor([sample_by_label(labels, value) for value in sudoku_p[nonzero]])
        vizsudoku[nonzero] = images[idxs]
        return vizsudoku
    
#vizsudoku = sample_visual_sudoku(sudoku_p0, testloader)

In [ ]:
# sudoku's, from http://hakank.org/minizinc/sudoku_problems2/index.html

sudoku_p0 = torch.IntTensor([[0,0,0, 2,0,5, 0,0,0],
                             [0,9,0, 0,0,0, 7,3,0],
                             [0,0,2, 0,0,9, 0,6,0],
                             [2,0,0, 0,0,0, 4,0,9],
                             [0,0,0, 0,7,0, 0,0,0],
                             [6,0,9, 0,0,0, 0,0,1],
                             [0,8,0, 4,0,0, 1,0,0],
                             [0,6,3, 0,0,0, 0,8,0],
                             [0,0,0, 6,0,8, 0,0,0]])
                            


In [ ]:
from ortools.sat.python import cp_model

# model and solve a sudoku with ortools
def model_sudoku_ort(grid):
        csp = cp_model.CpModel()

        # init vars
        board = [[csp.NewIntVar(1, 9, 'x_%i%i' % (i,j)) for j in range(9)] for i in range(9)]
        
        # assign knowns
        for i in range(9):
            for j in range(9):
                if grid[i,j] != 0:
                    csp.Add(board[i][j] == grid[i,j].item())
        
        # all different rows
        for i in range(9):
            csp.AddAllDifferent(board[i])
        
        # all different columns
        for j in range(9):
            csp.AddAllDifferent([board[i][j] for i in range(9)])
        
        # all different cells
        for si in range(3):
            for sj in range(3):
                csp.AddAllDifferent([board[3*si+i][3*sj+j] for j in range(3) for i in range(3)])
        
        return csp, board
def solve_sudoku_ort(grid):
    # the constraint model and decision vars
    csp, x = model_sudoku_ort(grid)
    
    solver = cp_model.CpSolver()
    status = solver.Solve(csp) # or similar?
    solution = np.zeros((9,9))
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE :

        for i in range(9):
            for j in range(9):
                solution[i,j] = int(solver.Value(x[i][j]))
        return solution 

sol = solve_sudoku_ort(sudoku_p0)
sol

## Finding the maximum likelihood solution

As errors in the output may lead to infeasible sudoku's, we are going to want to find the _maximum likelihood_ solution.

First, we read and store the prediction probabilities instead of the predictions. We obtain a 9x9x9 tensor (last dimension = probabilities of digit 1..9)


In [ ]:
# get probabilities of predictions
def predict_proba_sudoku(model, vizsudoku):
    # reshape from 9x9x28x28 to 81x1x28x28
    pred = model(vizsudoku.flatten(0,1))
    # our NN return 81 probabilistic vector: an 81x10 matrix
    return pred.reshape(9,9,10).detach() # reshape as 9x9x19 tensor for easier visualisation


## Maximum likelihood estimation with standard CP solver

We need to turn the _satisfaction_ problem of sudoku into an _optimisation_ problem, where we optimize for maximum log likelihood.

__Task: adapt the above code to find the maximum likelihood visual sudoku solution!__

This means adding the objective function: a weighted sum of the decision variables, with as weight the log-probability of that decision variable being equal to the corresponding predicted value.

E.g. $\sum_i \sum_j \sum_c log(prob[i,j,c])*[V[i,j] == c]$

Note that the only thing that changes is adding the objective, so you can reuse model_sudoku_ort() of an empty grid!!

We assume that cells containing given clues are available through a $is\_given$ boolean matrix. 



In [ ]:
from ortools.sat.python import cp_model

def solve_vizsudoku_ort(probs, is_given):
    # the constraint model
    empty_grid = torch.zeros((9,9), dtype=torch.int)
    csp, x = model_sudoku_ort(empty_grid)
    
    # TODO: add the objective function!!
    
    solver = cp_model.CpSolver()
    status = solver.Solve(csp) 
    solution = np.zeros((9,9))
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE :
        for i in range(9):
            for j in range(9):
                solution[i,j] = int(solver.Value(x[i][j]))
        return solution 

is_given = sudoku_p0 > 0
vsudoku = sample_visual_sudoku(sudoku_p0, testloader)

probs = predict_proba_sudoku(model, vsudoku)
is_given
psol = solve_vizsudoku_ort(probs, is_given)
psol

# Visualizing prediction error

Our trained Neural network classifies an image correctly if it assigns the highest score to the true label. Thus, we can assess the accuracy of the model by comparing maximum likelihood labels against labels in the numerical instance. 

In [ ]:
ml_digits = np.argmax(probs.numpy(), -1) 

fig, axes = plt.subplots(9, 9, figsize=(1.5*9,2*9))
for i in range(9*9):
    
    ax = axes[i//9, i%9]
    c = 'gray' if ml_digits.reshape(-1)[i] == sudoku_p0.reshape(-1)[i] else 'autumn'
    if not is_given.reshape(-1)[i]:
        # ignore cell with zero display black square instead
        ax.imshow(torch.zeros(28,28).float(), cmap='gray')
        ax.set_axis_off()
        continue
    ax.imshow(vsudoku.view(-1,28,28)[i].squeeze(), cmap=c)

    # replace 0 with blanks and don't show their label  
    ax.set_title('Label: {}'.format(ml_digits.reshape(-1)[i]))
    ax.set_axis_off()

In [ ]:
## fetch images to fill empty cells
digit_indices = {k:np.where(testset.targets == k) for k in range(1,10)}
digit_supply = {k:len(v[0]) for k,v in digit_indices.items()}
imgs_supply = {k:testset.data[digit_indices[k]] for k in range(1,10)}

##helper function to plot and compare solution found with hybrid approach
def plot_vs(visualsudoku, output, is_given, ml_digits, solution):
    n = 9
    fig, axes = plt.subplots(n, n, figsize=(1.5*n,2*n))

    for i in range(n*n):
        ax = axes[i//n, i%n]
        # sample image wrt solver output
        img = torch.zeros(28,28).float()
        c = 'gray'
        if not is_given.reshape(-1)[i]:
            # cell filled by the solver in gray
            img = imgs_supply[output.reshape(-1)[i]][0]
        else:
            img = visualsudoku.view(-1, 28,28)[i].squeeze()
            # wrong given -> red
            # given fixed by cp -> green
            c = 'gray' if output.reshape(-1)[i] == ml_digits.reshape(-1)[i] else 'summer'

        c = 'autumn' if output.reshape(-1)[i] != solution.reshape(-1)[i] else c

        if c == 'summer':
            ax.set_title('ML label: {}\nsolver label: {}'.format(ml_digits.reshape(-1)[i], output.reshape(-1)[i]))
        elif c == 'autumn':
            ax.set_title('solver label: {}\nTrue label: {}'.format(output.reshape(-1)[i], solution.reshape(-1)[i]))
            
        ax.imshow(img, cmap=c)
        ax.set_axis_off()

In [ ]:
plot_vs(vsudoku, psol, is_given, ml_digits, sol)